In [1]:
using Pkg 
Pkg.instantiate()
using QAlgebra

using ComplexRationals
using LaTeXStrings

In [2]:
qspace = StateSpace("alpha", "beta_i(t)", "gamma_i", "delta_i", operators=["A(i)", "B(U,H)"], h=QubitPM(), i=(3, QubitPauli()), b=Ladder())

StateSpace: [βᵢ(t), βⱼ(t), βₖ(t), γᵢ, γⱼ, γₖ, δᵢ, δⱼ, δₖ, α]
   - SubSpace ["h"]: PM Qubit (Fermionic):  pₚ, mₚ, zₚ, Iₚ (identity)
   - SubSpace ["i", "j", "k"]: Pauli Qubit (Fermionic):  xₚ, yₚ, zₚ, Iₚ (identity)
   - SubSpace ["b"]: Ladder (Bosonic):  p†, p
   - Op: A
   - Op: B(H,U)


In [3]:
#a = FAtom(1, [1,0])
#b = FAtom(2, [1,2])
a = FAtom(ComplexRational(1,0,1), [1,0])
b = FAtom(ComplexRational(2,0,1), [1,2])
s = simplify(a/(a+b))
s = to_string(a/(-a*b-a), ["\\alpha", "\\beta"], do_latex=true, do_frac=true)
latexstring(s)

L"$-\frac{1}{1+2 \alpha\beta^{2}}$"

In [4]:
my_sign, my_str = to_stringer(-2*a*b-4*b, ["\\alpha", "\\beta(t)"], do_latex=false, braced=true)
latexstring(my_str)

L"$4\alpha\beta(t)²(2+\alpha)$"

In [5]:
my_sign, my_str = to_stringer(-(1+2im)*b*a^(-2), ["\\alpha", "\\beta(t)"], do_latex=true, braced=true, do_frac=true)
latexstring(my_str)

L"$\frac{(2 + 4i) \beta(t)^{2} }{\alpha}$"

In [ ]:
function simple_combinable_F(t1::Union{FAtom, FSum}, t2::Union{FAtom, FSum})::Tuple{Bool, ComplexRational}
    # Transform FAtom's to FSums 
    if t1 isa FAtom
        t1 = FSum([t1])
    end
    if t2 isa FAtom
        t2 = FSum([t2])
    end
    @assert length(t1) == length(t2) "Cannot separate pair terms with different lengths."

    # check if corresponding terms have the similar exponents (i.e. the difference of exponents has to be the same for each term in FSum)
    exponents_diff = a1.terms[1].var_exponents .- a2.terms[1].var_exponents
    if !all([diff == 0 for diff in exponents_diff])
        return false, ComplexRational(0, 0, 0)
    end
    for (a1, a2) in zip(t1.terms[2:end], t2.terms[2:end])
        if a1.var_exponents .- a2.var_exponents != exponents_diff
            return false, ComplexRational(0, 0, 0)
        end
    end
    
    # check if the terms have a constant ratio in the coefficients 
    ratio = t1.terms[1].coeff / t2.terms[1].coeff
    if !(ratio.b == 0 || ratio.a == 0)
        return false, ratio
    end
    for (a1, a2) in zip(t1.terms[2:end], t2.terms[2:end])
        if a1.coeff / a2.coeff != ratio
            return false, ratio
        end
    end
    return true, ratio
end
# check for multiple elements in a Vector of FSum 
function simple_combinable_Fs(ts::Vector{Union{FAtom,FSum}})::Tuple{Vector{Vector{Union{FAtom, FSum}}}, Vector{Vector{Int}}}
    groups = Vector{Vector{Union{FAtom,FSum}}}()
    indexes = Vector{Vector{Int}}()
    for (i, t) in enumerate(ts)
        placed = false
        for (inds, grp) in zip(indexes, groups)
            ok,_ = simple_combinable_F(grp[1], t)
            if ok 
                push!(grp, t)
                push!(inds, i)
                placed = true
                break
            end
        end
        if !placed
            push!(groups, [t])
            push!(indexes, [i])
        end
    end
    return groups, indexes
end

function ratios_Fs(ts::Vector{Union{FAtom,FSum}})::Vector{ComplexRational}
    if ts[1] isa FAtom 
        c1 = ts[1].coeff
    else 
        c1 = ts[1].terms[1].coeff
    end
    ratios = ComplexRational[ComplexRational(1,0,1)]
    for t in ts[2:end] 
        if t isa FAtom 
            c2 = t.coeff
        else 
            c2 = t.terms[1].coeff
        end
        push!(ratios, c1/c2)
    end
    return ratios
end
function group_Fs(ts::Vector{Union{FAtom,FSum}})::Union{FAtom,FSum, Tuple{FAtom, Vector{Union{FAtom, FSum}}}}
    # find the correct way to group a group of Fs (they must be groupable, create the input vector with simple_combinable_Fs)
    if length(ts) == 1
        return ts[1]
    elseif length(ts) > 1
        # we need to find the best common ratio 
        ratios = ratios_Fs(ts)
        base, multiples = common_denominator_form(ratios)
        pre_F = ts[1]*base 
        post_Fs = [ t*m for (t,m) in zip(ts, multiples)]
        return (pre_F, post_Fs)
    end
end

"""
    how_to_combine_Fs(ts::Vector{Union{FAtom, FSum}}) :: Tuple{Vector{Union{FAtom, FSum, Tuple{FAtom, Vector{Union{FAtom, FSum}}}}}, Vector{Vector{Int}}}

Groups and combines `FAtom` and `FSum` objects in the input vector `ts` into composite structures that can be processed together. 
Returns a tuple containing both the groups as (FFunction) elements of a Vector and the indexs corresponding to the elements in the groups. 
The (FFunction) grouping is given either by a single FFunction element (either a single `FAtom` or a `FSum`) or by a tuple of an `FAtom` (F1) containing the shared factors, and a vector of FFunction elements (F2_i), so that 
together they represent a term of the form: F1 * (F2_1 + ... + F2_n). 
"""
function how_to_combine_Fs(ts::Vector{Union{FAtom,FSum}})::Tuple{Vector{Union{FAtom,FSum, Tuple{FAtom, Vector{Union{FAtom, FSum}}}}}, Vector{Vector{Int}}}
    if length(ts) == 1
        return [ts[1]], [[1]]
    elseif length(ts) > 1 
        groups, indexes = simple_combinable_Fs(ts)
        return [group_Fs(grp) for grp in groups], indexes
    else
        return [], []
    end
end

how_to_combine_Fs (generic function with 1 method)

In [ ]:
function QComposite2string(q::QAtomProduct; do_latex::Bool=true, do_sigma::Bool=false, braced::Bool=true, do_frac::Bool=true)::Tuple{Bool, String}
    num = is_numeric(q)
    if is_numeric
        curr_sign, curr_str = to_stringer(q.coeff_fun, variable_str_vec(q, do_latex=do_latex), braced=braced, do_frac=do_frac)
        return curr_sign, curr_str
    else
        curr_sign, curr_str = to_stringer(q.coeff_fun, variable_str_vec(q, do_latex=do_latex), braced=braced, do_frac=do_frac)
        operator_str = join([qAtom2string(t, q.statespace, do_latex=do_latex, do_sigma=do_sigma) for t in q.expr], "")
        connector = do_latex ? raw"\cdot" : "*"
        if printnumeric(q.coeff_fun)
            return curr_sign, curr_str * connector * operator_str
        else
            return curr_sign, operator_str 
        end
    end
end

function QComposites2string(terms::Vector{QComposite}; do_latex::Bool=false, do_sigma::Bool=false, braced::Bool=true, do_frac::Bool=true, separate_sign::Bool=false)::String
    substrings::Vector{{Tuple{Bool, String}} = [QComposite2string(t, do_latex=do_latex, do_sigma=do_sigma, braced=braced, do_frac=do_frac) for t in terms]
    # connect substrings  
    if separate_sign
        string = substrings[1][2]
    else
        string = substrings[1][1] ? "-" * substrings[1][2] : substrings[1][2]
    end
    for (curr_sign, curr_str) in substrings[2:end]
        string *= curr_sign ? "-" * curr_str : "+" * curr_str
    end
    if separate_sign 
        return substrings[1][1], string
    end
    return string
end
import .FFunction: how_to_combine_Fs
function group_qAtomProducts(qs::Vector{QAtomProduct})::Vector{Union{QAtomProduct, Tuple{Union{FAtom, FSum}, Vector{QAtomProduct}}}
    coeffs_funs = [q.coeff_fun for q in qs]
    coeff_groups, indexes = how_to_combine_Fs(coeffs_funs)
    new_qs = []
    for (coeffs, indexes) in zip(coeff_groups, indexes)
        if !( coeff isa Tuple )
            push!(new_qs, copy(qs[indexes[1]]))
        else
            pre_F = coeffs[1]
            post_F = coeffs[2]
            post_q = QAtomProduct[]
            for (F, i) in zip(post_F, indexes) 
                new_p = copy(qs[i]) 
                new_p.coeff_fun *= F 
                push!(post_q, new_p) 
            end 
            push!(new_qs, (pre_F, post_q)) 
        end
    end 
    return new_qs 
end 

function qAtomProduct_group2string(qs::QAtomProduct; do_latex::Bool=true, do_sigma::Bool=false, braced::Bool=true, do_frac::Bool=true)::String
    curr_sign, operator_str = QComposite2string(qs[1], do_latex=do_latex, do_sigma=do_sigma, braced=braced, do_frac=do_frac)
    return curr_sign, "", operator_str
end
function qAtomProduct_group2string(qs::Tuple{Union{FAtom, FSum}, Vector{QAtomProduct}; do_latex::Bool=true, do_sigma::Bool=false, braced::Bool=true, do_frac::Bool=true)::String
    # assume the qs can be simple grouped (see the functions: simple_combinable_Fs, group_Fs)
    F = qs[1]
    qs = qs[2]
    f_sign, f_str = to_stringer(F; do_latex=do_latex, braced=braced, do_frac=do_frac)
    q_str = QComposites2string(qs; do_latex=do_latex, do_sigma=do_sigma, braced=braced, do_frac=do_frac, separate_sign=false)  # don't worry about internal signs, this has already been taken care off by the sign handling of the grouping 
    return f_sign, f_str, q_str
end 

function allnegative(x::Tuple{Bool, String})::Bool 
    return x[1]
end
function allnegative(x::Vector{Tuple{Bool, String}})::Bool
    return all(allnegative, x)
end
function brace(x::String; do_latex::Bool=true)::String
    if do_latex 
        return raw"\left(" * x * raw"\right)"
    else
        return "(" * x * ")"
    end
end

function QExpr2string(q::QExpr; do_latex::Bool=true, do_sigma::Bool=false, braced::Bool=true, do_frac::Bool=true, do_grouped::Bool=true, return_grouping::Bool=false)::Union{Tuple{Bool, String}, Tuple{Bool, String, Bool}}
    # outputs sign, string, {optional return_grouping:} single_group::Bool   => return grouping implies that the expression will be braced if it isn't already! , hence the outputted sign is handled differently 
    # first sort terms 
    q_sorted = sort(q)
    if !do_grouped # outside (not inside of a QComposite)
        if !return_grouping
            return QComposites2string(q_sorted.terms, do_latex=do_latex, do_sigma=do_sigma, braced=braced, do_frac=do_frac, separate_sign=true)
        else
            return QComposites2string(q_sorted.terms, do_latex=do_latex, do_sigma=do_sigma, braced=braced, do_frac=do_frac, separate_sign=true), false
    else  # iunside amnother QComposite
        # separate into QAtomProduct and other QComposite terms -> sorting puts qAtomProducts first 
        # then group qAtomProducts by their factors 
        first_non_qAtomProduct = findfirst(x -> !isa(x, QAtomProduct), q_sorted.terms)
        if first_non_qAtomProduct === nothing
            first_non_qAtomProduct = length(q_sorted.terms)
        end
        qAtomProduct_terms = q_sorted.terms[1:first_non_qAtomProduct]
        other_terms = q_sorted.terms[first_non_qAtomProduct+1:end]
        groups = group_qAtomProducts(qAtomProduct_terms)
        # create strings for each element 
        all_strings = []
        for group in groups
            curr_sign, first, second = qAtomProduct_group2string(group, do_latex=do_latex, do_sigma=do_sigma, braced=braced, do_frac=do_frac)
            push!(all_strings, (curr_sign, first*brace(second, do_latex=do_latex))
        end
        for term in other_terms
            push!(all_strings, QComposite2string(term, do_latex=do_latex, do_sigma=do_sigma, braced=braced, do_frac=do_frac))
        end
        
        # make QComposites2string better, by allowing a third output in case of single group. to traverse multiple layers of Composites within composites. 
        if return_grouping 
            # do we switch the sign? 
            if allnegative(all_strings) || ( get_default(:FLIP_IF_FIRST_TERM_NEGATIVE) && all_strings[1][1] )
                # switch signs 
                first_sign = true 
                all_strings = [(!sign, s) for (sign, s) in all_strings]
            else
                first_sign = false
            end
        else
            first_sign = all_strings[1][1]
        end
        total_string = all_strings[1][1] ? "-" * all_strings[1][2] : all_strings[1][2]
        for (sign, s) in all_strings[2:end]
            if sign 
                total_string *= "-" * s
            else 
                total_string *= "+" * s
            end
        end
        if return_grouping 
            single_group::Bool = length(groups) == 1 && length(all_strings) == 1
            return first_sign, total_string, single_group
        else
            return first_sign, total_string
        end
    end
end



QComposite2string (generic function with 1 method)

In [15]:
sort([((1,2),[1,2]), ((3,4), [12,2])])

2-element Vector{Tuple{Tuple{Int64, Int64}, Vector{Int64}}}:
 ((1, 2), [1, 2])
 ((3, 4), [12, 2])

In [ ]:
const DEFAULT_COEFF_PREFS = Dict(
    :FLOAT_DIGITS => 2,
    :EXP_DIGITS => 2,
    :FLIP_IF_FIRST_TERM_NEGATIVE => true,
    )

function get_default(name::Symbol)
    return DEFAULT_COEFF_PREFS[name]
end

get_default (generic function with 1 method)

In [4]:
using ComplexRationals
abstract type FFunction end

mutable struct FAtom      <: FFunction
    coeff::ComplexRational
    var_exponents::Vector{Int}
    function FAtom(coeff::Int, var_exponents::Vector{Int})
        c = ComplexRational(coeff, 0, 1)
        return new(c, var_exponents)
    end
    function FAtom(coeff::Rational, var_exponents::Vector{Int})
        c = ComplexRational(numerator(coeff), 0, denominator(coeff))
        return new(c, var_exponents)
    end
    function FAtom(coeff::ComplexRational, var_exponents::Vector{Int})
        return new(coeff, var_exponents)
    end
end

mutable struct FSum       <: FFunction
    terms::Vector{FFunction}
    # (inner) constructor for a Vector{<:FFunction>, with flattening
    function FSum(ts::AbstractVector{<:FFunction})
        flat = FFunction[]
        for t in ts
            if t isa FSum
                append!(flat, (t::FSum).terms)   # flatten nested sums
            else
                push!(flat, t)
            end
        end
        new(flat)
    end
end

# varargs “outer” constructor so you can call FSum(a,b,c) directly
FSum(ts::FFunction...) = FSum(collect(ts))

mutable struct FRational  <: FFunction
    numer::FSum
    denom::FSum
end

_terms(f::FFunction) = f isa FSum ? (f::FSum).terms : [f]

import Base: isless
function isless(a::FAtom, b::FAtom)
    a.var_exponents < b.var_exponents    # Vector{Int} has lex order
end


import Base: iszero, isempty

iszero(a::FAtom)        = iszero(a.coeff)
iszero(s::FSum)         = isempty(s.terms) || all(iszero, s.terms)
iszero(r::FRational)    = iszero(r.numer)

isempty(s::FSum)        = isempty(s.terms)

# true if no variables (all exponents zero)
isnumeric(a::FAtom)    = all(e->e==0, a.var_exponents)
isnumeric(s::FSum)     = all(isnumeric, s.terms)
isnumeric(r::FRational)= isnumeric(r.numer) && isnumeric(r.denom)

allnegative(a::FAtom) = is_negative(a.coeff)
allnegative(s::FSum)  = !isempty(s.terms) && all(allnegative, s.terms)
allnegative(r::FRational) = allnegative(r.numer)

min_exponents(a::FAtom)    = a.var_exponents
function min_exponents(s::FSum)
    if length(s.terms) == 0
        return []
    end
    min_vals = min_exponents(s.terms[1])
    for term in s.terms[2:end]
        curr_min_vals = min_exponents(term)
        min_vals = min.(min_vals, curr_min_vals)
    end
    return min_vals
end
function min_exponents(r::FRational)
    min_vals = min_exponents(r.numer)
    min_vals2 = min_exponents(r.denom)
    return min.(min_vals, min_vals2)
end


import Base: length, getindex, iterate, deleteat!, reverse
function length(p::FSum)::Int
    return length(p.terms)
end
getindex(p::FSum, i::Int) = p.terms[i]
iterate(p::FSum, state=1) = state > length(p.terms) ? nothing : (p.terms[state], state + 1)
deleteat!(p::FSum, i::Int) = FSum(deleteat!(p.terms, i))
reverse(q::FSum) = FSum(reverse(q.terms))
dims(q::FAtom) = length(q.var_exponents)
dims(q::FSum) = dims(q.terms[1])
dims(q::FRational) = dims(q.numer) 

import Base: +, -, *, /, ^, ==

# addition always builds a flat sum
+(a::FFunction) =  a 
+(a::FFunction, b::FFunction) = FSum( vcat(_terms(a), _terms(b)) )

# unary minus & subtraction
import Base: -, +

-(a::FAtom) = FAtom(-a.coeff, a.var_exponents)
-(s::FSum) = FSum([ -t for t in s.terms ])
-(r::FRational) = FRational(-r.numer, r.denom)
-(a::FFunction, b::FFunction) = a + (-b)


# distribute * over sums
*(A::FSum, B::FSum)       = FSum([ x*y for x in A.terms,   y in B.terms ])
*(A::FSum, b::FFunction)  = FSum([ x*b for x in A.terms ])
*(a::FFunction, B::FSum)  = FSum([ a*y for y in B.terms ])

# atom‐level ×
*(a::FAtom, b::FAtom)     = FAtom(crationalize(a.coeff*b.coeff), a.var_exponents .+ b.var_exponents)
*(a::FAtom, r::FRational) = FRational(FSum(a)*r.numer, r.denom)
*(r::FRational, a::FAtom) = FRational(r.numer*FSum(a), r.denom)
*(a::FRational, b::FRational) = FRational(a.numer*b.numer, a.denom*b.denom)
*(a::FSum, b::FRational) = FRational(a*b.numer, b.denom)
*(b::FRational, a::FSum) = FRational(a*b.numer, b.denom)
# number 
*(a::FFunction, b::Number)  = a*FAtom(b, zeros(Int, dims(a)))
*(b::Number, a::FFunction)  = FAtom(b, zeros(Int, dims(a))) * a
multiply_one(a::FRational, b::Int) = (a.numer * b) / (a.denom * b)

# division
/(A::FSum, B::FSum) = length(B.terms)==1 ? FSum([ x/B.terms[1] for x in A.terms ]) : FRational(A, B)
/(A::FAtom, B::FSum)     = FRational(FSum(A), B)
/(A::FSum, b::FAtom)     = FSum([ x/b for x in A.terms ])
/(a::FAtom, b::FAtom)     = FAtom(a.coeff/b.coeff, a.var_exponents .- b.var_exponents)
/(a::FAtom, r::FRational) = FRational(FSum(a)*r.denom, r.numer)
/(r::FRational, a::FAtom) = FRational(r.numer, r.denom*FSum(a))
/(a::FRational, b::FRational) = FRational(a.numer*b.denom, a.denom*b.numer)
/(a::FRational, b::FSum) = FRational(a.numer, b.numer*b)
/(a::FSum, b::FRational) = FRational(a*b.denom, b.numer)
/(a::FFunction, n::Number) = a/FAtom(n, zeros(Int, dims(a)))
/(n::Number, a::FFunction) = FAtom(n, zeros(Int, dims(a))) / a

# exponentiation 
^(A::FSum, n::Int) = FSum([ x^n for x in A.terms ])
^(A::FAtom, n::Int) = FAtom(A.coeff^n, A.var_exponents .* n)
^(a::FRational, n::Int) = FRational(a.numer^n, a.denom^n)

function ==(a::FAtom, b::FAtom)
    return (a.coeff == b.coeff && a.var_exponents == b.var_exponents)
end
function ==(a::FSum, b::FSum)
    if length(a) != length(b)
        return false
    end
    for i in 1:length(a) 
        if a[i] != b[i]
            return false
        end
    end
    return true
end
function ==(a::FRational, b::FRational)
    return (a.numer == b.numer && a.denom == b.denom)
end

function unifiable(a::FAtom, b::FAtom)::Bool
    return a.var_exponents == b.var_exponents
end
function unifiable(a::FRational, b::FRational)::Bool
    return a.denom == b.denom
end
function unifiable(a::FSum, b::FSum)::Bool
    true
end
# assume inifiable
function unify(a::FAtom, b::FAtom)::FFunction
    absum = a.coeff+b.coeff
    if iszero(absum)
        var_zeros = zeros(Int, dims(a))
        return FAtom(0, var_zeros)
    end
    return FAtom(absum, a.var_exponents)
end
function unify(a::FRational, b::FRational)::FFunction
    simple_numer = simplify(a.numer+b.numer)
    if iszero(simple_numer)
        var_zeros = zeros(Int, dims) 
        return FAtom(0, var_zeros)
    end
    return FRational(simple_numer, a.denom)
end
function unify(a::FSum, b::FSum)::FFunction
    return simplify(a+b)
end

unify (generic function with 3 methods)

In [5]:
import Base: sort!, sort

function complexrational2coeffs(a::ComplexRational)::Vector{Int}
    return [a.a, a.b, a.c]
end

function sort_key(a::FAtom)
    return vcat(a.var_exponents, complexrational2coeffs(a.coeff))
end

function sort_key(s::FSum)
    keys = [sort_key(t) for t in s.terms]
    return vcat(length(keys), keys...)
end
function sort_key(r::FRational)
    dkeys = sort_key(r.denom)
    nkeys = sort_key(r.numer)
    return vcat(length(dkeys), dkeys, nkeys)
end

function sort!(s::FSum; by=sort_key)
    sort!(s.terms, by=by)
end
function sort!(s::FRational, by=sort_key)
    sort!(s.numer.terms, by=by)
    sort!(s.denom.terms, by=by)
end
function sort!(s::FAtom; by=sort_key)
    # do nothing 
    return s
end

function sort(s::FSum; by=sort_key)::FSum
    return FSum(sort(s.terms, by=by))
end
function sort(s::FRational; by=sort_key)::FRational
    numer = sort(s.numer.terms, by=by)
    denom = sort(s.denom.terms, by=by)
    return FRational(numer, denom)
end
function sort(s::FAtom; by=sort_key)::FAtom
    # do nothing 
    return s
end

# Test 
a = FAtom(2, [0,1])
b = FAtom(3, [1,0])
c = FAtom(4, [1,1])
d = FSum([a,c,b,c])
sort(d)

FSum(FFunction[FAtom(2, [0, 1]), FAtom(3, [1, 0]), FAtom(4, [1, 1]), FAtom(4, [1, 1])])

In [6]:
issimple(f::FFunction) = error("Not implemented for type $(typeof(f)).")
issimple(f::FSum) = all(t -> t isa FAtom, f.terms)
issimple(f::FRational) = issimple(f.numer) && issimple(f.denom)
issimple(f::FAtom) = true


issimple (generic function with 4 methods)

In [5]:
# divisors 
function divisors(a::FFunction)
    error("Not implemented for type $(typeof(a)).")
end
function divisors(a::FAtom)::Vector{Int}
    return [a.coeff.c]
end
function divisors(a::FSum)::Vector{Int}
    return reduce(vcat, [divisors(t) for t in a.terms])
end
function divisors(a::FRational)::Vector{Int}
    return reduce(vcat, [divisors(t) for t in [a.numer, a.denom]])
end
# simplify rationals by multiplying the 

divisors (generic function with 4 methods)

In [6]:
function vec_multiply(x::FAtom, vector::Vector{Int})::FAtom
    return FAtom(x.coeff, x.var_exponents - vector)
end
function vec_multiply(x::FSum, vector::Vector{Int})::FSum
    return FSum([vec_multiply(t, vector) for t in x.terms])
end
function vec_multiply(x::FRational, vector::Vector{Int})::FRational
    return FRational(vec_multiply(x.numer), vec_multiply(x.denom))
end

vec_multiply (generic function with 3 methods)

In [7]:
# Assumes that the divisors are 1 
function coeffs(s::FAtom)::Vector{Int}
    # if subtype complex take real and imaginary parts separately 
    return [s.coeff.a, s.coeff.b]
end
function coeffs(s::FSum)::Vector{Int}
    return vcat([coeffs(t) for t in s.terms]...)
end
function coeffs(s::FRational)::Vector{Int}
    return vcat(coeffs(s.numer), coeffs(s.denom))
end
import Base: gcd
function gcd(s::FFunction)
    return gcd(coeffs(s))   
end
function gcd(s::FFunction, t::FFunction)
    return gcd(vcat(coeffs(s), coeffs(t)))
end
#gcd(a*2+b+a*4)

gcd (generic function with 17 methods)

In [ ]:
function simplify(s::FAtom)
    return s 
end
function simplify(r::FRational)
    n = simplify(r.numer)
    d = simplify(r.denom)
    # remove fractions in coefficients in Rational
    curr_div = vcat(divisors(n), divisors(d))
    factor = lcm(curr_div...)
    n = factor * n
    d = factor * d
    # common denominator
    factor = gcd(n, d)
    n = n / factor
    d = d / factor

    min_n = min_exponents(n)
    min_d = min_exponents(d)
    min_vals = min.(min_n, min_d)
    n = vec_multiply(n, min_vals)
    d = vec_multiply(d, min_vals)
    if allnegative(d) || (get_default(:FLIP_IF_FIRST_TERM_NEGATIVE) && allnegative(d[1]))   # prefer negatives on numerator
        n = -n
        d = -d
    end
    # if denom now has exactly one term, collapse back to a sum
    if length(d.terms) == 0
        error("Divinding by zero")
    elseif length(d.terms) == 1
        # use our /‐overload to divide each term in n by d.terms[1]
        return n / d.terms[1]
    else
        return FRational(n, d)
    end
end

function simplify(s::FSum)
    # first simplify the lower levels 
    elements = [simplify(e) for e in s.terms]
    s = FSum(elements)
    sort!(s)
    i = 1
    elements = s.terms
    new_elements = FFunction[]
    curr_element = elements[1]
    for i in 2:length(elements)
        if typeof(curr_element) == typeof(elements[i]) 
            if unifiable(curr_element, elements[i])
                curr_element = unify(curr_element, elements[i])
            else
                if !iszero(curr_element)
                    push!(new_elements, curr_element)
                end
                curr_element = elements[i]
            end
        else
            if !iszero(curr_element)
                push!(new_elements, curr_element)
            end
            curr_element = elements[i]
        end
    end
    if !iszero(curr_element)
        push!(new_elements, curr_element)
    end
    if length(new_elements) == 0 
        push!(new_elements, FAtom(0, zeros(Int, dims(s))))
    end
    return FSum(new_elements)
end


"""
    max_exponents(f::FFunction) -> Vector{Int}

For an FAtom: its own exponents.  
For an FSum: elementwise max over all terms.  
For an FRational: returns a pair `(max_numer, max_denom)`.
"""
max_exponents(a::FAtom) = abs.(a.var_exponents)
function max_exponents(s::FSum)
    if length(s.terms) == 0
        return []
    else
        max_m = max_exponents(s[1])
        for t in s.terms[2:end]
            curr_max = max_exponents(t)
            for i in eachindex(curr_max)
                if curr_max[i] > max_m[i]
                    max_m[i] = curr_max[i] 
                end
            end
        end
        return max_m
    end
end
function max_exponents(r::FRational)
    max_a = max_exponents(r.numer)
    max_b = max_exponents(r.denom)
    return [max(a, b) for (a, b) in zip(max_a, max_b)]
end
"""
    build_xpows(x::Vector{<:Number}, M::Vector{Int})

Returns a vector xpows of length `length(x)`, where
  xpows[j][k+1] = x[j]^k,  for k = 0:M[j].
"""
function build_xpows(x::Vector{<:Number}, max_exp::Vector{Int})::Vector{Vector}
    @assert length(x) == length(M)
    xpows = [Vector{typeof(x[i])}(undef, length(x)) for i in eachindex(x)]
    for j in eachindex(x)
        xpows[j] = [ x[j]^k for k in 0:max_exp[j] ]
    end
    return xpows
end


function evaluate(a::FAtom, x::AbstractVector{<:Number})
    a.coeff * prod(x[i]^a.var_exponents[i] for i in eachindex(a.var_exponents))
end
function evaluate(s::FSum, x::AbstractVector{<:Number})
    sum(evaluate(t, x) for t in s.terms)
end
function evaluate(r::FRational, x::AbstractVector{<:Number})
    evaluate(r.numer, x) / evaluate(r.denom, x)
end

# Evaluate but with xpows
function evaluate(a::FAtom, xpows::Vector{Vector})
    return a.coeff * prod(xpows[i][a.var_exponents[i]+1] for i in eachindex(a.var_exponents))
end
function evaluate(s::FSum, xpows::Vector{Vector})
    return sum(evaluate(t, xpows) for t in s.terms)
end
function evaluate(r::FRational, xpows::Vector{Vector})
    return evaluate(r.numer, xpows) / evaluate(r.denom, xpows)
end

evaluate (generic function with 6 methods)

In [12]:
function sign_string(c::ComplexRational, do_latex::Bool=false)::Tuple{Bool, String}
    if is_negative(c)
        return (true, string(c, do_latex=do_latex)[2:end])
    else
        return (false, string(c, do_latex=do_latex))
    end
end
is_abs_one(c::ComplexRational)::Bool = (abs(c.a) == abs(c.b))
function is_abs_one(c::FFunction)
    if isnumeric(c)
        if isa(c, FAtom)
            return is_abs_one(c.coeff)
        elseif isa(c, FSum)
            if length(c) == 1
                return is_abs_one(c[1])
            else
                return false
            end
        else
            return false
        end
        return true
    else
        return false
    end
end

is_abs_one (generic function with 2 methods)

In [ ]:
# --- Generic string constructor ---
function stringer(f::FFunction)::Tuple{String,String}
    error("No fallback method for FFunction type: $(typeof(f))")
end

# --- FAtom ---
function stringer(a::FAtom)::Tuple{Bool,String}  # true = minus, false = plus
    if isnumeric(a)
        return sign_string(a.coeff) 
    else
        c = a.coeff
        vec = string(a.var_exponents)
        sign, c_str = sign_string(c) 
        if is_abs_one(c)
            return (sign, vec)
        else
            return sign,  c_str*"*"*vec
        end
    end
end

# --- FSum ---
function stringer(s::FSum; braced::Bool=false) ::Tuple{Bool,String}
    # braced specifies whether the terms will be grouped, so that an external sign is needed
    s = simplify(s)
    terms = s.terms
    if braced 
        if allnegative(s) || (get_default(:FLIP_IF_FIRST_TERM_NEGATIVE) && allnegative(s[1]))
            # if all negative and braced, we can just negate the whole thing
            sig = true
            _, body = stringer(-s)
            return sig, body
        else
            sig = false
            body = join(stringer(t) for t in terms)
        end
    end

    # process each term into (sign, body)
    parts = String[]
    for (i, t) in enumerate(terms)
        sig, body = stringer(t)
        if i == 1
            # first term keeps its sign, but no space if positive
            push!(parts, sig == true ? "-$body" : body)
        else
            push!(parts, sig == true ? "-$body" : "+$body")
        end
    end
    return false, join(parts, "")
end


# --- FRational ---
function stringer(r::FRational)::Tuple{Bool,String}
    r = simplify(r)
    n = r.numer
    d = r.denom

    # check first term in numerator
    n_sig, n_body = stringer(n, braced=true)
    d_sig, d_body = stringer(d)  # we ignore sign of denom

    if length(n) > 1
        n_body = "($n_body)"
    end
    if length(d) > 1
        d_body = "($d_body)"
    end

    return n_sig, "$n_body/$d_body"
end

# --- Show ---
import Base: show

function show(io::IO, f::FFunction)
    sig, body = stringer(f)
    sig_str = sig ? "-" : ""
    print(io, sig_str * body)
end



a = FAtom(1, [1,0])
b = FAtom(2, [0,1])
p = FSum([a, b])
s = simplify(a/(a*b+a))
#evaluate(s, [2,3])
#max_exponents(s)

1/(1+2*[0, 1])

In [15]:
const superscript_indexes = Dict('a' => "ᵃ", 'b' => "ᵇ", 'c' => "ᶜ", 'd' => "ᵈ", 'e' => "ᵉ", 'f' => "ᶠ",
    'g' => "ᵍ", 'h' => "ʰ", 'i' => "ⁱ", 'j' => "ʲ", 'k' => "ᵏ", 'l' => "ˡ", 'm' => "ᵐ", 'n' => "ⁿ",
    'o' => "ᵒ", 'p' => "ᵖ", 'q' => "ᵠ", 'r' => "ʳ", 's' => "ˢ", 't' => "ᵗ", 'u' => "ᵘ", 'v' => "ᵛ",
    'w' => "ʷ", 'x' => "ˣ", 'y' => "ʸ", 'z' => "ᶻ", '2' => "²", '3' => "³", '4' => "⁴", '5' => "⁵", '6' => "⁶", '7' => "⁷", '8' => "⁸", '9' => "⁹", '1' => "", '-' => "⁻", '=' => "⁼")

function str2sup(s::String)::String
    new_str = ""
    for c in s
        if haskey(superscript_indexes, Char(c))
            new_str *= superscript_indexes[Char(c)]
        else
            #@warn "Character $c not found in superscript_indexes, printing as ^$c instead. Avoid this by choosing one of the following characters: $superscript_indexes.keys()"
            new_str *= "^$c"
        end
    end
    return new_str
end

str2sup (generic function with 1 method)

In [19]:
# Generic fallback
function stringer(f::FFunction, vars::Vector{String}; do_latex::Bool=false)
    error("No stringer method for type $(typeof(f)) with variable names")
end

# --- FAtom with variable names ---
function stringer(a::FAtom, vars::Vector{String}; do_latex::Bool=false)
    exps = a.var_exponents
    @assert length(vars) == length(exps) "Number of symbols must match number of variables"

    if isnumeric(a)
        return sign_string(a.coeff) 
    else
        # build the variable part
        varparts = String[]
        for (i, e) in enumerate(exps)
            if e == 0
                continue
            elseif do_latex
                push!(varparts, e == 1 ? vars[i] : "$(vars[i])^{$e}")
            else
                push!(varparts, e == 1 ? vars[i] : "$(vars[i])"*str2sup(string(e)))
            end
        end

        var_str = isempty(varparts) ? "" : join(varparts, do_latex ? "\\cdot " : "")
        c = a.coeff
        sign, c_str = sign_string(c) 
        if is_abs_one(c)
            return (sign, vec)
        else
            return sign,  c_str*"*"*var_str
        end
    end
end

# --- FSum with variable names ---
function stringer(s::FSum, vars::Vector{String}; do_latex::Bool=false, braced::Bool=false)
    s = simplify(s)
    terms = s.terms
    if isempty(terms)
        return false, "0"
    end

    parts = String[]
    for (i, t) in enumerate(terms)
        sig, body = stringer(t, vars; do_latex=do_latex)
        if i == 1
            push!(parts, sig ? "-" * body : body)
        else
            push!(parts, sig ? "-" * body : "+" * body)
        end
    end

    out = join(parts, "")
    return false, out
end

# --- FRational with variable names ---
function stringer(r::FRational, vars::Vector{String}; do_latex::Bool=false)
    r = simplify(r)
    n = r.numer
    d = r.denom

    n_sig, n_str = stringer(n, vars; do_latex=do_latex, braced=true)
    _, d_str = stringer(d, vars; do_latex=do_latex, braced=true)

    if do_latex
        return n_sig, "\\frac{$n_str}{$d_str}"
    else
        n_wrapped = length(n.terms) > 1 ? "($n_str)" : n_str
        d_wrapped = length(d.terms) > 1 ? "($d_str)" : d_str
        return n_sig, "$n_wrapped/$d_wrapped"
    end
end

"""
    to_string(f::FFunction, vars::Vector{String};
              do_latex::Bool = false,
              braced::Bool = false) -> String

Converts the FFunction `f` into a string using variable names from `vars`.

If `do_latex=true`, uses LaTeX syntax (e.g., `\\frac{}` and `x^2`).  
If `braced=true`, wraps the expression in `()` or `{}` if it's a sum.

The result includes the correct sign (`-`) if the top-level expression is negative.
"""
function to_stringer(f::FFunction, vars::Vector{String}; do_latex::Bool=false, braced::Bool=false)::Tuple{Bool, String}
    if braced && f isa FSum && length(f) > 1
        sig, body = stringer(f, vars; do_latex=do_latex, braced=braced)
    else
        sig, body = stringer(f, vars; do_latex=do_latex)
    end
    # Apply braces to sums if requested
    if braced && f isa FSum && length(f) > 1
        body = do_latex ? "\\left( $body \\right)" : "($body)"
    end
    return sig, body
end
function to_string(f::FFunction, vars::Vector{String}; do_latex::Bool=false, braced::Bool=false, optional_sign::Bool=true)::String
    sig, body = to_stringer(f, vars; do_latex=do_latex, braced=braced)

    if sig
        return "-" * body
    else
        if optional_sign
            return body
        end
        return "+" *body
    end
end

#a = FAtom(1, [1,0])
#b = FAtom(2, [1,2])
a = FAtom(ComplexRational(1,0,1), [1,0])
b = FAtom(ComplexRational(2,0,1), [1,2])
s = simplify(a/(a+b))
s = to_string(a/(a*b+a), ["x", "y"], do_latex=false)

"1/(1+2*xy²)"

In [ ]:
s = to_string(a/(a*b+a), ["\\alpha", "\\beta"], do_latex=true)
using LaTeXStrings
latexstring(s)

L"$\frac{1}{1+2*\alpha\cdot \beta^{2}}$"